In [ ]:
import requests
import re
import numpy as np
import json
import pandas as pd

In [ ]:
corp = []

In [ ]:
with open("gyakori_szamitastechnika_line", "r+") as f:
    for i,line in enumerate(f):
        obj = json.loads(line.strip("\n"))
        corp.append(obj)

In [ ]:
df = pd.DataFrame(corp)

In [ ]:
df["kategoriak"] = df.kategoriak.apply(lambda x: x[1])

In [ ]:
df.kategoriak

In [ ]:
p = df.groupby("kategoriak").size()

In [ ]:
p.plot.pie(figsize=(5, 5))

In [ ]:
df.hosszu_kerdes.str.len()

In [ ]:
from sklearn.model_selection import train_test_split as split

In [ ]:
type(df.hosszu_kerdes)

In [ ]:
import hu_core_ud_lg
import tqdm

In [ ]:
from tqdm import tqdm
from gensim.models.keyedvectors import KeyedVectors

In [ ]:
hu_nlp = hu_core_ud_lg.load()

In [ ]:
def preprocess_text(data):
    print(len(data))
    clean_data = []
    for i,kerdes in tqdm(enumerate(data), "Preprocessing"):
        doc = hu_nlp(data[i])
        clean_data.append([tok.lemma_ for tok in doc if tok.is_alpha and not tok.is_stop])
    return clean_data

In [ ]:
def vectorize(tr_data, tst_data):
    print('\nLoading existing FastText model...')
    model = KeyedVectors.load_word2vec_format("./embeddings/hu.szte.w2v.fasttext.vec.2")
    vectorizer = model.wv
    vocab_length = len(model.wv.vocab)
    """
    tr_vectors = [
        np.array([vectorizer[word] for word in tweet if word in model]).flatten() for tweet in tqdm(tr_data,'Vectorizing')
        ]
    max_len = np.max([len(vector) for vector in tr_vectors])
    tr_vectors = [
        np.array(vector.tolist()+[0 for _ in range(max_len-len(vector))]) for vector in tqdm(tr_vectors,'Finalizing')
        ]
    
    tst_vectors = [
        np.array([vectorizer[word] for word in tweet if word in model]).flatten() for tweet in tqdm(tst_data,'Vectorizing')
        ]

    tst_vectors = [
        np.array(vector.tolist()+[0 for _ in range(max_len-len(vector))]) for vector in tqdm(tst_vectors,'Finalizing')
        ]
    """
    
    tr_vectors = [
        np.array(np.mean([vectorizer[word] for word in tweet if word in model], axis=0)) for tweet in tqdm(tr_data,'Vectorizing')
    ]
    
    tst_vectors = [
        np.array(np.mean([vectorizer[word] for word in tweet if word in model], axis=0)) for tweet in tqdm(tst_data,'Vectorizing')
    ]
    
    return tr_vectors, tst_vectors

In [ ]:
def get_features_and_labels(data, labels):
    tr_data,tst_data,tr_labels,tst_labels = split(data,labels,test_size=0.3)
    tr_data_clean = preprocess_text(tr_data)
    tst_data_clean = preprocess_text(tst_data)
    
    tst_vecs = []
    tr_vecs = []
    tr_vecs, tst_vecs = vectorize(tr_data_clean, tst_data_clean)    
    return tr_vecs, tr_labels, tst_vecs, tst_labels

In [ ]:
tr_vecs, tr_labels, tst_vecs, tst_labels = get_features_and_labels(df.hosszu_kerdes.tolist(), df.kategoriak.tolist())

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
model = LinearSVC()

In [ ]:
model.fit(tr_vecs, tr_labels)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(tst_vecs)

print("Test accuracy : {}".format(accuracy_score(tst_labels, y_pred)))

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(tst_labels, y_pred))

In [ ]:
df.groupby("kategoriak").size()

In [ ]:
for i in tst_vecs:
    print(len(i))

In [ ]:
with open("gyakori_egeszseg_10000", "r+") as f:
    data = f.read()
    eg_corp = json.loads(data)

In [179]:
eg = pd.DataFrame.from_dict(eg_corp)

In [180]:
eg.head()

,kategoriak,valasz,hosszu_kerdes,rovid_kerdes,keywords
0,"[Egészség, Sérülések, balesetek]","Komolyan az eszem megáll! Azt, hogy leforrázta...",Ma sajnos leforráztam a kisbabám lábát! Úgy go...,Ma sajnos leforráztam a kisbabám lábát! Úgy go...,"[baba, forró víz, leforrázás]"
1,"[Egészség, Mentális egészség]",Nekem sokat segített a pszichológus ebben a té...,Mi a neve ennek a mentális betegségnek? A pszi...,Mi a neve ennek a mentális betegségnek? A pszi...,"[pszichológia, pszichológus, lélek, elme, fóbi..."
2,"[Egészség, Férfiak egészsége]","Miért lenne baj, hogy ""még van fitymád""? A fér...",Az a baj ha 15 évesen még van fitymám és hogy ...,Az a baj ha 15 évesen még van fitymám és hogy ...,"[fityma, egészség, pénisz, betegség, normális,..."
3,"[Egészség, Betegségek]",Legyél 1 srác és minden megjavul.,26 srác vagyok minden reggel hányinger és néha...,26 srác vagyok minden reggel hányinger és néha...,"[pánikbetegség, hányinger, hasmenés]"
4,"[Egészség, Fogak, szájápolás]","Jaja, véletlen se az orvosra hallgass, aki 6+ ...",Húzassam ki? Vagy gyökérkezelés? A bal legháts...,Húzassam ki? Vagy gyökérkezelés?,"[fog, gyökérkezelés]"


In [181]:
eg.head()

,kategoriak,valasz,hosszu_kerdes,rovid_kerdes,keywords
0,"[Egészség, Sérülések, balesetek]","Komolyan az eszem megáll! Azt, hogy leforrázta...",Ma sajnos leforráztam a kisbabám lábát! Úgy go...,Ma sajnos leforráztam a kisbabám lábát! Úgy go...,"[baba, forró víz, leforrázás]"
1,"[Egészség, Mentális egészség]",Nekem sokat segített a pszichológus ebben a té...,Mi a neve ennek a mentális betegségnek? A pszi...,Mi a neve ennek a mentális betegségnek? A pszi...,"[pszichológia, pszichológus, lélek, elme, fóbi..."
2,"[Egészség, Férfiak egészsége]","Miért lenne baj, hogy ""még van fitymád""? A fér...",Az a baj ha 15 évesen még van fitymám és hogy ...,Az a baj ha 15 évesen még van fitymám és hogy ...,"[fityma, egészség, pénisz, betegség, normális,..."
3,"[Egészség, Betegségek]",Legyél 1 srác és minden megjavul.,26 srác vagyok minden reggel hányinger és néha...,26 srác vagyok minden reggel hányinger és néha...,"[pánikbetegség, hányinger, hasmenés]"
4,"[Egészség, Fogak, szájápolás]","Jaja, véletlen se az orvosra hallgass, aki 6+ ...",Húzassam ki? Vagy gyökérkezelés? A bal legháts...,Húzassam ki? Vagy gyökérkezelés?,"[fog, gyökérkezelés]"


In [182]:
eg["kategoriak"] = eg.kategoriak.apply(lambda x: x[1])

In [183]:
eg.head()

,kategoriak,valasz,hosszu_kerdes,rovid_kerdes,keywords
0,"Sérülések, balesetek","Komolyan az eszem megáll! Azt, hogy leforrázta...",Ma sajnos leforráztam a kisbabám lábát! Úgy go...,Ma sajnos leforráztam a kisbabám lábát! Úgy go...,"[baba, forró víz, leforrázás]"
1,Mentális egészség,Nekem sokat segített a pszichológus ebben a té...,Mi a neve ennek a mentális betegségnek? A pszi...,Mi a neve ennek a mentális betegségnek? A pszi...,"[pszichológia, pszichológus, lélek, elme, fóbi..."
2,Férfiak egészsége,"Miért lenne baj, hogy ""még van fitymád""? A fér...",Az a baj ha 15 évesen még van fitymám és hogy ...,Az a baj ha 15 évesen még van fitymám és hogy ...,"[fityma, egészség, pénisz, betegség, normális,..."
3,Betegségek,Legyél 1 srác és minden megjavul.,26 srác vagyok minden reggel hányinger és néha...,26 srác vagyok minden reggel hányinger és néha...,"[pánikbetegség, hányinger, hasmenés]"
4,"Fogak, szájápolás","Jaja, véletlen se az orvosra hallgass, aki 6+ ...",Húzassam ki? Vagy gyökérkezelés? A bal legháts...,Húzassam ki? Vagy gyökérkezelés?,"[fog, gyökérkezelés]"


In [187]:
p = eg.groupby("kategoriak").size()

In [194]:
data = eg.hosszu_kerdes.tolist()
labels = eg.valasz.tolist()

with open("gyakori_seq2seq", "w+", errors='surrogatepass') as f:
    for line in zip(data, labels):
        f.write(line[0] + "\t" + line[1] + "\n")

In [221]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /userhome/student/adaamko/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [220]:
# Tokenized input
text = "[CLS] Komolyan?? én ezt nem hiszem, el."
tokenized_text = tokenizer.tokenize(text)
tokenized_text

['[CLS]',
 'komo',
 '##lya',
 '##n',
 '?',
 '?',
 'en',
 'ezt',
 'nem',
 'his',
 '##zem',
 ',',
 'el',
 '.']

In [222]:
import hu_core_ud_lg

nlp = hu_core_ud_lg.load()
doc = nlp('Csiribiri csiribiri zabszalma - négy csillag közt alszom ma.')

In [235]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator, TabularDataset

import spacy

import random
import math
import time

In [227]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [232]:
def tokenize_hu(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in nlp(text)]

In [233]:
SRC = Field(tokenize = tokenize_hu, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_hu, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [243]:
train, val = TabularDataset.splits(path=".",
        format='csv', skip_header=True,
        train='train.tsv', validation='dev.tsv',
        fields=[('src',SRC), ("trg",TRG)])

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 5392: invalid continuation byte

In [ ]:
with open("gyakori_seq2seq", "r+") as f:
    for line in f:
        f.write(line[0] + "\t" + line[1] + "\n")